In [3]:
import yaml
from sklearn.cross_validation import train_test_split
import multiprocessing
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout,Activation
from keras.models import model_from_yaml
np.random.seed(1337)  # For Reproducibility
import jieba
import pandas as pd
import sys
sys.setrecursionlimit(1000000)
vocab_dim = 300
maxlen = 100
n_iterations = 1  # ideally more..
n_exposures = 10
window_size = 5
batch_size = 32
n_epoch = 4
input_length = 100
cpu_count = multiprocessing.cpu_count()

In [4]:
#加载训练文件并分词
#加载训练文件
def loadfile():
    neg=pd.read_excel('data/neg.xls',header=None,index=None)
    pos=pd.read_excel('data/pos.xls',header=None,index=None)

    combined=np.concatenate((pos[0], neg[0]))
    y = np.concatenate((np.ones(len(pos),dtype=int), np.zeros(len(neg),dtype=int)))

    return combined,y

#对句子经行分词，并去掉换行符
def tokenizer(text):
    ''' Simple Parser converting each document to lower-case, then
        removing the breaks for new lines and finally splitting on the
        whitespace
    '''
    text = [jieba.lcut(document.replace('\n', '')) for document in text]
    return text



创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引

In [5]:
def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                            allow_update=True)
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量

        def parse_dataset(combined):
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0)
                data.append(new_txt)
            return data
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print('No data provided...')


#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def word2vec_train(combined):

    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count)
    model.build_vocab(combined)
    model.train(combined,total_examples=model.corpus_count,epochs=model.epochs)
    model.save('lstm_data/Word2vec_model.pkl')
    index_dict, word_vectors,combined = create_dictionaries(model=model,combined=combined)
    return   index_dict, word_vectors,combined

训练网络，并保存模型，其中LSTM的实现采用Python中的keras库

In [6]:
def get_data(index_dict,word_vectors,combined,y):

    n_symbols = len(index_dict) + 1  # 所有单词的索引数，频数小于10的词语索引为0，所以加1
    embedding_weights = np.zeros((n_symbols, vocab_dim))#索引为0的词语，词向量全为0
    for word, index in index_dict.items():#从索引为1的词语开始，对每个词语对应其词向量
        embedding_weights[index, :] = word_vectors[word]
    x_train, x_test, y_train, y_test = train_test_split(combined, y, test_size=0.2)
    print(x_train.shape,y_train.shape)
    return n_symbols,embedding_weights,x_train,y_train,x_test,y_test


##定义网络结构
def train_lstm(n_symbols,embedding_weights,x_train,y_train,x_test,y_test):
    print('Defining a Simple Keras Model...')
    model = Sequential()  # or Graph or whatever
    model.add(Embedding(output_dim=vocab_dim,
                        input_dim=n_symbols,
                        mask_zero=True,
                        weights=[embedding_weights],
                        input_length=input_length))  # Adding Input Length
    model.add(LSTM(output_dim=50, activation='sigmoid', inner_activation='hard_sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    print('Compiling the Model...')
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',metrics=['accuracy'])

    print("Train...")
    model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=n_epoch,verbose=1, validation_data=(x_test, y_test))

    print("Evaluate...")
    score = model.evaluate(x_test, y_test,
                                batch_size=batch_size)

    yaml_string = model.to_yaml()
    with open('lstm_data/lstm.yml', 'w') as outfile:
        outfile.write( yaml.dump(yaml_string, default_flow_style=True) )
    model.save_weights('lstm_data/lstm.h5')
    print('Test score:', score)


#训练模型，并保存


In [7]:
import time
tic = time.time()
print('Loading Data...')
combined,y=loadfile()
print(len(combined),len(y))
print('Tokenising...')
combined = tokenizer(combined)

Loading Data...


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jason\AppData\Local\Temp\jieba.cache


21105 21105
Tokenising...


Loading model cost 0.828 seconds.
Prefix dict has been built succesfully.


In [8]:
print('Training a Word2vec model...')
index_dict, word_vectors,combined=word2vec_train(combined)
print('Setting up Arrays for Keras Embedding Layer...')



Training a Word2vec model...


c:\users\jason\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Setting up Arrays for Keras Embedding Layer...


In [9]:
n_symbols,embedding_weights,x_train,y_train,x_test,y_test=get_data(index_dict, word_vectors,combined,y)
print(x_train.shape,y_train.shape)

(16884, 100) (16884,)
(16884, 100) (16884,)


In [10]:
train_lstm(n_symbols,embedding_weights,x_train,y_train,x_test,y_test)
toc = time.time()
print("cost=",toc-tic)

Defining a Simple Keras Model...


c:\users\jason\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=50, activation="sigmoid", recurrent_activation="hard_sigmoid")`


Compiling the Model...
Train...


c:\users\jason\appdata\local\programs\python\python35\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 16884 samples, validate on 4221 samples
Epoch 1/4
16884/16884 [==============================] - 55s 3ms/step - loss: 0.5322 - acc: 0.7192 - val_loss: 0.3129 - val_acc: 0.8813
Epoch 2/4
16884/16884 [==============================] - 55s 3ms/step - loss: 0.2592 - acc: 0.9065 - val_loss: 0.2306 - val_acc: 0.9149
Epoch 3/4
16884/16884 [==============================] - 55s 3ms/step - loss: 0.1592 - acc: 0.9502 - val_loss: 0.2264 - val_acc: 0.9211
Epoch 4/4
16884/16884 [==============================] - 54s 3ms/step - loss: 0.1145 - acc: 0.9643 - val_loss: 0.2426 - val_acc: 0.9173
Evaluate...
4221/4221 [==============================] - 3s 713us/step
Test score: [0.2426456199653113, 0.9173181710777563]
cost= 305.402067899704
